In [61]:
import pandas as pd
import numpy as np
import datetime
import string
import os
from matplotlib import pyplot as plt
from pathlib import Path

In [62]:
DATA_PATH = '/Users/vuhoangnguyen/Research/'
RAW_DATA = os.path.join(DATA_PATH, 'raw_data.csv')

In [63]:
dataset = pd.read_csv(RAW_DATA, keep_default_na=False)

In [64]:
dataset = dataset.rename(columns={"GIOI.x": "Female"})
s = dataset['Female'].to_numpy()

for i in range(len(s)):
    if s[i] == 'Nam' or s[i] == 'M':
        dataset.at[i,'Female'] = 0
    elif s[i] == 'Nu' or s[i] == 'F':
        dataset.at[i,'Female'] = 1

In [65]:
dataset = dataset.drop(columns=['STT', 'ID', 'Name', 'SODIENTHOAI', 'Fisrt of Department',
                                'Department of out hospital', 'Department of test'])

In [66]:
month_name = ['Jan', 'Feb', 'Mar', 'Apr',
              'May', 'Jun', 'Jul', 'Aug',
              'Sep', 'Oct', 'Nov', 'Dec']

def to_datetime(column_name):
    s = dataset[column_name].to_numpy(dtype = 'str')
    for i in range(len(s)):
        if s[i] == 'nan':
            continue
        if '-' in s[i]:
            date = s[i].split(sep='-')
        elif '/' in s[i]:
            date = s[i].split(sep='/')
        for j in range(len(month_name)):
            if date[1] == month_name[j]:
                date[1] = j+1
        date = [int(x) for x in date]
        if date[0] > 1000: # date[0] is year
            x = datetime.date(date[0],date[1],date[2])
        else:
            if date[2] < 1000: # date[2] is year
                date[2] += 2000
            x = datetime.date(date[2],date[1],date[0])
        dataset.at[i,column_name] = x

dataset = dataset.rename(columns={"NGAYSINH.x":"dob"})
to_datetime('dob')
dataset = dataset.rename(columns={"date of admission":"admission_date"})
to_datetime('admission_date')

In [67]:
dataset = dataset.rename(columns={"date of onset":"onset_date"})
onset_date_str = dataset['onset_date'].to_numpy(dtype = 'str')

for i in range(len(onset_date_str)):
    current_date = onset_date_str[i]
    if current_date == 'nan':
        continue
    if '-' in current_date:
        date = current_date.split(sep='-')
    if '/' in current_date:
        date = current_date.split(sep='/')
    for j in range(len(month_name)):
        if date[1] == month_name[j]:
            date[1] = j+1
    date = [int(x) for x in date]
    if date[2] < 1000: # date[2] is year
        date[2] += 2000
    x = datetime.date(date[2],date[1],date[0])
    dataset.at[i,'onset_date'] = x

In [68]:
# correct row 1085 (21/7/209) to 21/7/2019

for i in range(dataset.shape[0]):
    d = dataset.at[i,'onset_date'].day
    m = dataset.at[i,'onset_date'].month
    y = dataset.at[i,'onset_date'].year
    if y not in [2017,2018,2019,2014]:
        dataset.at[i,'onset_date'] = datetime.date(2019, m, d)

In [69]:
dataset.insert(7, "age_at_admission", 0)

In [70]:
for i in range(dataset.shape[0]):
    x = dataset.at[i,'dob']
    y = dataset.at[i,'admission_date']
    if pd.isna(dataset.at[i,'dob']) == False and pd.isna(dataset.at[i,'admission_date']) == False:
        z = y - x
        dataset.at[i,"age_at_admission"] = z.days // 30

In [71]:
dataset = dataset.rename(columns={"Distric":"district"})
district = dataset['district'].to_numpy(dtype='str')

wrong_district = ["Huyen V_ Quang","ph_ c_","Huyen Hi_p H","Kim B_ng","Huyen H__ng Kh?","Nam Tr_c","Th__ng Xu","T__ng D__ng","Huyen C_ Kuin",
"B_m S_n","Y?n Khnh","Bnh Xuy?n","Huyen Kim __ng","Xun Tr__ng","Tam D__ng","Huyen Qu_nh L_u","Thi Xa Thai Ho?a","Thi Xa Thai Ho`a",
"TP B_c Giang","L__ng S_n","Y?n L_c","Ung B","Huyen Nng C_ng","Kim Thnh","Huyen Di_n Ch","T_ k_","Gia Vi_n","Giao Th_y","TP. Thi Nguy?n",
"L_p Th_ch","L_c Nam","Huyen T_ K_","tp L_ng S_n","tp. Ninh Bnh","Vi_t Tr","Huyen C_m Kh?","Phc Thnh","B_c Quang","Huyen Thanh Ch__ng",
"H_ H","Huyen Bnh Giang","Mai S_n","Huyen Tam Nâ¢ng","Th_y Nguy?n","Tp Thi Bnh","Ngh_a H_ng","_inh H","Thnh ph_ H_i D__ng","TP. Ninh Bnh",
"Ch Linh","TP. Nam __nh","TP.H_ng Y?n","Kinh M_n","Thanh Mi_n","H_ng Nguy?n","Nam Sch","Huyen Ngh_a _","H_u L_c","Nam c__ng","Quang S_n",
"Phc Y?n","Gia Bnh","Huyen _ng H_ng","Y?n M_","ån Thi","V_nh t__ng","Huyen V_n L","L_c H","__ S_n","H Phong","__ng T"]
correct_district = ["Huyen Vu Quang","Phu Cu","Huyen Hiep Hoa","Kim Bang","Huyen Huong Khe","Nam Truc","Thuong Xuan","Tuong Duong","Huyen Cu Kuin",
"Bim Son","Yen Khanh","Binh Xuyen","Huyen Kim Dong","Xuan Truong","Tam Duong","Huyen Quynh Luu","Thi Xa Thai Hoa","Thi Xa Thai Hoa",
"TP Bac Giang","Luong Son","Yen Lac","Uong Bi","Huyen Nong Cong","Kim Thanh","Huyen Dien Chau","Tu ky","Gia Vien","Giao Thuy","TP Thai Nguyen",
"Lap Thach","Luc Nam","Huyen Tu Ky","TP Lang Son","TP Ninh Binh","Viet Tri","Huyen Cam Khe","Phuc Thanh","Bac Quang","Huyen Thanh Chuong",
"Ha Hoa","Huyen Binh Giang","Mai Son","Huyen Tam Nong","Thuy Nguyen","TP Thai Binh","Nghia Hung","Dinh Hoa","TP Hai Duong","TP Ninh Binh",
"Chi Linh","TP Nam Dinh","TP Hung Yen","Kinh Mon","Thanh Mien","Hung Nguyen","Nam Sach","Huyen Nghia Dan","Hau Loc","Nam Cuong","Quang Son",
"Phuc Yen","Gia Binh","Huyen Dong Hung","Yen My","An Thi","Vinh Tuong","Huyen Van Lam","Loc Ha","Do Son","Hong Phong","Dong Tam"]

for i in range(len(district)):
    if district[i] == 'nan':
        dataset.at[i,'district'] = ''
        continue
    for j in range(len(wrong_district)):
        if district[i] == wrong_district[j]:
            district[i] = correct_district[j]
            break
    word_dis = district[i].split(sep = ' ')
    if word_dis[0] == 'Huy_n':
        word_dis[0] = 'Huyen'
    if word_dis[0] == 'TX':
        word_dis[0] = 'Thi Xa'
    new_dis = ' '.join(word_dis)
    dataset.at[i,'district'] = new_dis

In [72]:
dis = dataset['district'].to_numpy(dtype='str')

for i in range(len(dis)):
    if dis[i] == '':
        continue
    dis[i] = string.capwords(dis[i])
    dataset.at[i,'district'] = dis[i]

In [73]:
dataset = dataset.rename(columns={"Province": "province"})
province = dataset['province'].to_numpy(dtype='str')

for i in range(len(dis)):
    if province[i] == 'nan':
        continue
    # correction for row 1106
    if province[i] == ']':
        province[i] = 'Nghe An'
    province[i] = string.capwords(province[i])
    # remove TP prefix for 'Tp Can Tho'
    province[i] = province[i].replace('Tp ','')
    dataset.at[i,'province'] = province[i]

In [74]:
dataset = dataset.rename(columns={"Community": "commune"})
comm = dataset['commune'].to_numpy(dtype='str')

for i in range(len(comm)):
    if comm[i] == 'nan':
        continue
    s = ""
    for j in comm[i]:
        if (j>='a' and j<='z') or (j>='A' and j<='Z') or j == ' ' or (j>='0' and j<='9'):
            s += j
    dataset.at[i,'commune'] = s

In [75]:
comm = dataset['commune'].to_numpy(dtype='str')

for i in range(len(comm)):
    if comm[i] == '':
        continue
    comm[i] = string.capwords(comm[i])
    dataset.at[i,'commune'] = comm[i]

In [76]:
dataset.insert(5,"complete_patient_address",np.nan)

In [77]:
for i in range(dataset.shape[0]):
    commune_i = dataset.at[i,'commune']
    district_i = dataset.at[i,'district']
    province_i = dataset.at[i,'province']
    address = []
    if commune_i != '':
        address.append(commune_i)
    if district_i != '':
        address.append(district_i)
    if province_i != '':
        address.append(province_i)
    full_address = ', '.join(address)
    dataset.at[i,'complete_patient_address'] = full_address

In [78]:
dataset = dataset.rename(columns = {"date of out hospital":"discharge_date"})
to_datetime('discharge_date')

In [79]:
dataset.insert(10,'duration_of_stay',np.nan)

In [80]:
for i in range(dataset.shape[0]):
    x = dataset.at[i,'admission_date']
    y = dataset.at[i,'discharge_date']
    if pd.isna(x) == False and pd.isna(y) == False:
        z = y - x
        dataset.at[i,"duration_of_stay"] = z.days

In [81]:
dataset = dataset.drop(columns = ['LOAIXN','Note of time stay in hospital'])

In [82]:
dataset = dataset.rename(columns = {'date of test':'test_date','date of result':'testresult_date'})
to_datetime('test_date')
to_datetime('testresult_date')

In [83]:
dataset = dataset.rename(columns = {'HAI':'healthcare_associated_infection','RSV':'respiratory_syncytical_virus','ADENO':'adenovirus',
'Pertussis':'pertussis','oxygen':'oxygen_cannula','cipap':'CPAP','ventilator':'conventional_mechanical_ventilation',
'HFO ventilator':'hfo_ventilation','ECMOY:AL':'ECMO'})

def YN_to_binary(column_name):
    s = dataset[column_name].to_numpy(dtype='str')
    for i in range(len(s)):
        if s[i] == '' or s[i] == 'nan':
            dataset.at[i,column_name] = 0
            continue
        ls = s[i].lower()
        if ls == 'yes':
            dataset.at[i,column_name] = 1
        elif ls == 'no' or ls == 'na':
            dataset.at[i,column_name] = 0

YN_to_binary('oxygen_cannula')
YN_to_binary('CPAP')
YN_to_binary('pertussis')
YN_to_binary('healthcare_associated_infection')
YN_to_binary('respiratory_syncytical_virus')
YN_to_binary('adenovirus')
YN_to_binary('conventional_mechanical_ventilation')
YN_to_binary('hfo_ventilation')
YN_to_binary('ECMO')

In [84]:
from datetime import date
def datetime_to_date(column_name):
    for i in range(dataset.shape[0]):
        x = dataset.at[i,column_name]
        y = pd.to_datetime(x)
        z = y.date()
        dataset.at[i,column_name] = z

datetime_to_date('dob')
datetime_to_date('admission_date')
datetime_to_date('discharge_date')
datetime_to_date('test_date')
datetime_to_date('testresult_date')

In [85]:
dataset = dataset.drop(columns = ['dialysis','Catheter','Onsetto out hospital','ghi chu nguon'])

In [86]:
dataset = dataset.rename(columns={'Code source': 'place_of_exposure'})

dataset['place_of_exposure'] = dataset['place_of_exposure'].str.lower()

for i in range(len(dataset['place_of_exposure'])):
    code = dataset.at[i, 'place_of_exposure']
    if code in ['suppected nch', 'definitely nch']:
        dataset.at[i, 'place_of_exposure'] = 'NHP'
    elif code in ['suppected other h', 'suppected  other h', 'definitely other h']:
        dataset.at[i, 'place_of_exposure'] = 'Another hospital'
    elif code in ['community']:
        continue

In [87]:
dataset.insert(20,'complication-gastroentiritis',0)

for i in range(dataset.shape[0]):
    x = dataset.at[i,'complication']
    if 'tieu chay' in x:
        dataset.at[i,'complication-gastroentiritis'] = 1

In [88]:
dataset.insert(21,'complication-middle-ear-infec',0)

for i in range(dataset.shape[0]):
    x = dataset.at[i,'complication']
    y = dataset.at[i,'Dianogis out of hospital']
    z = dataset.at[i,'Dianogis of test']
    if 'viem tai' in x or 'viem tai' in y or 'viem tai' in z:
        dataset.at[i,'complication-middle-ear-infec'] = 1

In [89]:
dataset.insert(22,'complication-conjunctivitis',0)

for i in range(dataset.shape[0]):
    x = dataset.at[i,'complication']
    y = dataset.at[i,'Dianogis out of hospital']
    z = dataset.at[i,'Dianogis of test']
    if 'ket mac' in x or 'ket mac' in y or 'ket mac' in z:
        dataset.at[i,'complication-conjunctivitis'] = 1

In [90]:
dataset.insert(23,'complication-laryngitis',0)

for i in range(dataset.shape[0]):
    x = dataset.at[i,'complication']
    y = dataset.at[i,'Dianogis out of hospital']
    z = dataset.at[i,'Dianogis of test']
    t = dataset.at[i,'Dianogis of admission']  
    if 'thanh quan' in x or 'thanh quan' in y or 'thanh quan' in z or 'thanh quan' in t:
        dataset.at[i,'complication-laryngitis'] = 1

In [91]:
dataset.insert(24,'complication-pneumonia-bronchitis',0)
key_word = ['viem phoi', 'viem phe quan','phe quan']

for i in range(dataset.shape[0]):
    x = dataset.at[i,'complication']
    y = dataset.at[i,'Dianogis out of hospital']
    z = dataset.at[i,'Dianogis of test']
    t = dataset.at[i,'Dianogis of admission']
    for word in key_word:
        if word in x or word in y or word in z or word in t:
            dataset.at[i,'complication-pneumonia-bronchitis'] = 1  
            break

In [92]:
dataset.insert(25,'complication-febrile-seizures',0)

for i in range(dataset.shape[0]):
    x = dataset.at[i,'complication']
    y = dataset.at[i,'Dianogis out of hospital']
    z = dataset.at[i,'Dianogis of test']
    t = dataset.at[i,'Dianogis of admission']
    if 'dong kinh' in x or 'dong kinh' in y or 'dong kinh' in z or 'dong kinh' in t:
        dataset.at[i,'complication-febrile-seizures'] = 1

In [93]:
dataset.insert(26,'complication-septic-shock-sepsis',0)
key_word = ['nhiem khuan huyet', 'soc nhiem khuan']

for i in range(dataset.shape[0]):
    x = dataset.at[i,'complication']
    y = dataset.at[i,'Dianogis out of hospital']
    z = dataset.at[i,'Dianogis of test']
    t = dataset.at[i,'Dianogis of admission']
    for word in key_word:
        if word in x or word in y or word in z or word in t:
            dataset.at[i,'complication-septic-shock-sepsis'] = 1

In [94]:
dataset.insert(27,'co-infection-influenza-a',0)
target_column = ['complication','Dianogis out of hospital','Dianogis of test','Dianogis of admission','Other disease']

for i in range(dataset.shape[0]):
    for j in target_column:
        x = dataset.at[i,j].lower()
        if 'cum a' in x:
            dataset.at[i,'co-infection-influenza-a'] = 1
            break

In [95]:
dataset.insert(28,'co-infection-influenza-b',0)
target_column = ['complication','Dianogis out of hospital','Dianogis of test','Dianogis of admission','Other disease']

for i in range(dataset.shape[0]):
    for j in target_column:
        x = dataset.at[i,j].lower()
        if 'cum b' in x:
            dataset.at[i,'co-infection-influenza-b'] = 1
            break

In [96]:
dataset.insert(29,'co-infection-streptococus-aerius',0)
target_column = ['complication','Dianogis out of hospital','Dianogis of test','Dianogis of admission','Other disease']

for i in range(dataset.shape[0]):
    for j in target_column:
        x = dataset.at[i,j].lower()
        if 'tu cau' in x:
            dataset.at[i,'co-infection-streptococus-aerius'] = 1
            break

In [97]:
dataset.insert(30,'co-infection-streptococus-pneumonia',0)
target_column = ['complication','Dianogis out of hospital','Dianogis of test','Dianogis of admission','Other disease']

for i in range(dataset.shape[0]):
    for j in target_column:
        x = dataset.at[i,j].lower()
        if 'phe cau' in x:
            dataset.at[i,'co-infection-streptococus-pneumonia'] = 1
            break

In [98]:
special_vaccine_status = ['','1 (12/2/2018)','1 (12/22/2018)','99','?a khoa qu?c t? H?i Phòng, ?i?u tr? ph?i','NA',
'd<U+1EE7> mui','kham bs lan anh','tiêm 15/9']
vaccine_value = ['NA',1,1,'NA','NA','NA',2,'NA',1]
vacc = dataset['Vaccination'].to_numpy(dtype=str)

for i in range(dataset.shape[0]):
    for j in range(len(special_vaccine_status)):
        if vacc[i] == special_vaccine_status[j]:
            vacc[i] = vaccine_value[j]
            break
    dataset.at[i,'Vaccination'] = vacc[i]

In [99]:
dataset.insert(39,'highest_ventilation_mode',0)

order = ['oxygen_cannula','CPAP','conventional_mechanical_ventilation','hfo_ventilation','ECMO']

for i in range(len(dataset)):
    for j in range(len(order)-1,-1,-1):
        if dataset.loc[i,order[j]] == 1:
            dataset.loc[i,'highest_ventilation_mode'] = order[j]
            break

In [100]:
stage_of_discharge_value = ['BN khong nhap vien','Bo ve','Chuyen vien','Ra vi<U+1EC7>n','Ra vien',
'Tu vong','Xin ve','chua ra vien','do giam','do, giam ','khoi','tu vong']
stage_of_discharge_new_value = ['khong nhap vien','bo ve','chuyen vien','ra vien','ra vien',
'tu vong','xin ve','chua ra vien','giam','giam','khoi','tu vong']

result_of_discharge_value = ['Chuyen  vien','Do/Giam','Khoi','Khong thay doi','Nang hon','Tu vong',
'chua ra vien','do giam','do, giam ','khoi','tu vong','Ð<U+1EE1>/Gi<U+1EA3>m']
result_of_discharge_new_value = ['chuyen vien','giam','khoi','khong thay doi','nang hon','tu vong',
'chua ra vien','giam','giam','khoi','tu vong','giam']

for i in range(len(dataset)):
    for j in range(len(stage_of_discharge_value)):
        if dataset.loc[i,'Stage of discharge'] == stage_of_discharge_value[j]:
            dataset.loc[i,'Stage of discharge'] = stage_of_discharge_new_value[j]
            break
    for j in range(len(result_of_discharge_value)):
        if dataset.loc[i,'result of discharge'] == result_of_discharge_value[j]:
            dataset.loc[i,'result of discharge'] = result_of_discharge_new_value[j]
            break

#stage_and_result = dataset[['Stage of discharge','result of discharge']]
#stage_and_result = stage_and_result.groupby(['Stage of discharge','result of discharge']).size().reset_index(name='counts')

dataset.insert(14,'outcome_died',0)

for i in range(len(dataset)):
    if dataset.loc[i,'result of discharge'] == 'tu vong':
        dataset.loc[i,'outcome_died'] = 1
    if dataset.loc[i,'Stage of discharge'] == 'tu vong':
        dataset.loc[i,'outcome_died'] = 1
    if dataset.loc[i,'Stage of discharge'] == 'xin ve' and dataset.loc[i,'result of discharge'] == 'nang hon':
        dataset.loc[i,'outcome_died'] = 1

dataset = dataset.drop(columns=['result of discharge','Stage of discharge'])

In [101]:
UPDATE_DATA = os.path.join(DATA_PATH, 'Duplicate.csv')
update_dataset = pd.read_csv(UPDATE_DATA, keep_default_na = True)
dataset.insert(6,'patient_home_coordinate',update_dataset['patient_home_coordinate'])
dataset.insert(7,'distance_to_nhp',update_dataset['distance_to_nhp'])

In [102]:
province = dataset['province'].to_numpy(dtype='str')
patient_per_province = {}

for i in province:
    if i in patient_per_province:
        patient_per_province[i] += 1
    else:
        patient_per_province[i] = 1
        
province_items = patient_per_province.items()
sorted_province = sorted(province_items, key=lambda province_items: province_items[0])
province_list = [i[0] for i in sorted_province]
patient_list = [i[1] for i in sorted_province]

In [103]:
part_of_vietnam = ['Ha Noi','Northeastern','Northwestern','Red River Delta (except Hanoi)','Central','Southern']

central_province = ['Thanh Hoa','Nghe An','Ha Tinh','Quang Binh','Quang Tri','Thua Thien Hue','Quang Nam','Quang Ngai',
                    'Binh Dinh','Phu Yen','Khanh Hoa','Ninh Thuan','Binh Thuan','Kon Tum','Gia Lai','Dak Lak']

south_province = ['Binh Phuoc','Binh Duong','Dong Nai','Tay Ninh','Ba Ria - Vung Tau','Ho Chi Minh','Long An',
                    'Dong Thap','Tien Giang','An Giang','Ben Tre','Can Tho']

northwestern_region = ['Lao Cai','Yen Bai','Lai Chau','Dien Bien','Son La','Hoa Binh']
northeastern_region = ['Ha Giang','Tuyen Quang','Phu Tho','Thai Nguyen','Bac Kan','Cao Bang','Lang Son','Bac Giang','Quang Ninh']
red_river_delta_except_hanoi = ['Hai Phong','Vinh Phuc','Bac Ninh','Hung Yen','Hai Duong','Thai Binh','Nam Dinh','Ninh Binh','Ha Nam']

#dataset.insert(6,'region_of_address',None)
for i in range(len(dataset)):
    if dataset.at[i,'province'] == 'Ha Noi':
        dataset.at[i,'region_of_address'] = part_of_vietnam[0]
    if dataset.at[i,'province'] in northeastern_region:
        dataset.at[i,'region_of_address'] = part_of_vietnam[1]
    if dataset.at[i,'province'] in northwestern_region:
        dataset.at[i,'region_of_address'] = part_of_vietnam[2]
    if dataset.at[i,'province'] in red_river_delta_except_hanoi:
        dataset.at[i,'region_of_address'] = part_of_vietnam[3]
    if dataset.at[i,'province'] in central_province:
        dataset.at[i,'region_of_address'] = part_of_vietnam[4]
    if dataset.at[i,'province'] in south_province:
        dataset.at[i,'region_of_address'] = part_of_vietnam[5]

In [104]:
underlying_condition = ['Underlying conditions - Respiratory system','Underlying conditions - Cardiovascular system',
    'Underlying condition - Gastrointestinal system','Underlying condition - Kidney and urology system',
    'Underlying condition - Immunodeficiency','Underlying condition - Neurological system','Underlying condition - Inherited metabolic disorders',
    'Underlying condition - No underlying diseases','Underlying condition - Other underlying conditions']

def chronic_diseases(insert_location,column_name,keywords):
    dataset.insert(insert_location,column_name,0)
    chronic = dataset['Chronic diseases code'].to_numpy(dtype='str')
    other = dataset['Other disease'].to_numpy(dtype='str')
    for i in range(len(dataset)):
        chronic_i = chronic[i].lower()
        other_i = other[i].lower()
        for key in keywords:
            if key in chronic_i or key in other_i:
                dataset.at[i,column_name] = 1
                break

chronic_diseases(19,'Underlying conditions - Respiratory system',keywords = ['ho hap','phoi'])
chronic_diseases(20,'Underlying conditions - Cardiovascular system',keywords = ['tim', 'mach', 'kawasaki'])
chronic_diseases(21,'Underlying condition - Gastrointestinal system',keywords=['chron', 'crohn', 'gan', 'mat', 'ruot', 'hau mon', 'tieu hoa', 'megacolon'])
chronic_diseases(22,'Underlying condition - Kidney and urology system', keywords=['than', 'bartter', 'tiet nieu'])
chronic_diseases(23,'Underlying condition - Immunodeficiency',keywords=['mddu', 'beuroblastome', 'suy giam mien dich'])
chronic_diseases(24,'Underlying condition - Neurological system',keywords=['than kinh', 'bai nao', 'bai noo', 'nao', 'noo', 'dong kinh', 'co giat'])
chronic_diseases(25,'Underlying condition - Inherited metabolic disorders',keywords=['roi loan chuyen hoa'])

def no_underlying_disease():
    column_name = 'Underlying condition - No underlying diseases'
    dataset.insert(26,column_name,0)
    chronic = dataset['Chronic diseases code'].to_numpy(dtype='str')
    other = dataset['Other disease'].to_numpy(dtype='str')
    for i in range(len(dataset)):
        chronic_i = chronic[i].lower()
        other_i = other[i].lower()
        if chronic_i == 'no' and other_i == 'no':
            dataset.at[i,column_name] = 1
    

def other_underlying_disease():
    column_name = 'Underlying condition - Other underlying conditions'
    dataset.insert(27,column_name,0)
    for i in range(len(dataset)):
        if sum([dataset.at[i,_] for _ in underlying_condition[:8]]) == 0:
            dataset.at[i,column_name] = 1
            
no_underlying_disease()
other_underlying_disease()

In [105]:
dataset = dataset.drop(columns=['stayhos','Dianogis of test','Dianogis out of hospital','Chronic diseases code','Other disease','complication'])

In [106]:
# swap columns so that admission_date, discharge_date, test_date, testresult_date, onset_date are next to each other
cols = dataset.columns.tolist()
cols[9], cols[10] = cols[10], cols[9] 
cols[13], cols[10] = cols[10], cols[13]
cols[14], cols[11] = cols[11], cols[14]
cols[45], cols[12] = cols[12], cols[45]
dataset = dataset[cols]

In [107]:
dataset.insert(51,'Clinical classification',0)
respiratory_support = ['oxygen_cannula','CPAP','conventional_mechanical_ventilation','hfo_ventilation','ECMO']

for i in range(len(dataset)):
    for j in respiratory_support:
        if dataset.loc[i,j] == 1:
            dataset.at[i,'Clinical classification'] = 1
            break

In [108]:
x = dataset['duration time onset to admission'].to_numpy()
for id,i in enumerate(x):
    if i == '#VALUE!':
        dataset.at[id,'duration time onset to admission'] = 'NA'

x = dataset['onset to test'].to_numpy()
for id,i in enumerate(x):
    if i == '#VALUE!':
        dataset.at[id,'onset to test'] = 'NA'

In [109]:
#Diagnosis upon admission: dựa trên cột diagnosis of admission
#- Measles: soi
#- Pneumonia: phoi
#- Bronchopneumonia: phe quan
#- Other diagnosis: còn lại

import re

dataset.insert(9,'Measles',0)
dataset.insert(10, 'Pneumonia',0)
dataset.insert(11, 'Bronchopneumonia',0)
dataset.insert(12, 'Other diagnosis',0)

dataset['Measles'] = np.where(dataset['Dianogis of admission'].str.contains('soi',flags=re.IGNORECASE), 1, 0)
dataset['Pneumonia'] = np.where(dataset['Dianogis of admission'].str.contains('phoi',flags=re.IGNORECASE), 1, 0)
dataset['Bronchopneumonia'] = np.where(dataset['Dianogis of admission'].str.contains('phe quan',flags=re.IGNORECASE), 1, 0)
dataset['Other diagnosis'] = np.where(dataset['Dianogis of admission'].str.contains('soi|phoi|phe quan',flags=re.IGNORECASE), 0, 1)

In [110]:
dataset.insert(57,'Age group (in months)',0)
dataset.insert(58,'Distance from the hospital (km)',0)
dataset.insert(59,'Duration between onset and admission',0)
dataset.insert(60,'Duration of stay within the hospital',0)
dataset.insert(61,'Duration between onset and test (detection time) (in hours)',0)

In [111]:
age_group_str = ['0 - 9','9 - 60','>= 60']
month_group = [0,9,60]
for i in range(len(dataset)):
    for j in range(len(month_group)-1,-1,-1):
        if dataset.at[i,'age_at_admission'] >= month_group[j]:
            dataset.at[i,'Age group (in months)'] = age_group_str[j]
            break

In [112]:
distance_marker = [0, 20, 200, 500]
distance_group = ['0 - <20','20 - <200','200 - <500','>= 500']
for i in range(len(dataset)):
    for j in range(len(distance_marker)-1,-1,-1):
        if dataset.at[i,'distance_to_nhp'] >= distance_marker[j]:
            dataset.at[i,'Distance from the hospital (km)'] = distance_group[j]
            break

In [113]:
onadmission_group = ['<0','0 - <3','3 - <7','7 - <14','>= 14']
onadmission_marker = [-10**9, 0, 3, 7, 14]

for i in range(len(dataset)):
    if dataset.at[i,'duration time onset to admission'] == 'NA':
        dataset.at[i,'Duration between onset and admission'] = 'Unknown'
        continue
    for j in range(len(onadmission_marker)-1,-1,-1):
        if int(dataset.at[i,'duration time onset to admission']) >= onadmission_marker[j]:
            dataset.at[i,'Duration between onset and admission'] = onadmission_group[j]
            break

In [114]:
duration_of_stay = dataset['duration_of_stay'].to_numpy()
duration_group = ['0 - <7','7 - <21','>= 21']
duration_marker = [0, 7, 21]

for i in range(len(dataset)):
    for j in range(len(duration_marker)-1,-1,-1):
        if duration_of_stay[i] >= duration_marker[j]:
            dataset.at[i,'Duration of stay within the hospital'] = duration_group[j]
            break

In [123]:
onsettest_group = ['< 0','0 - <24','24 - <48','>= 48']
onsettest_marker = [-10**9,0, 1, 2]    # 1 day = 24 hours

for i in range(len(dataset)):
    if i == 271:
        print(dataset.at[i,'onset to test'])
    if dataset.at[i, 'onset to test'] == 'NA':
        dataset.at[i,'Duration between onset and test (detection time) (in hours)'] = 'Unknown'
        #print(i)
        continue
    for j in range(len(onsettest_marker)-1,-1,-1):
        if int(dataset.at[i,'onset to test']) >= onsettest_marker[j]:
            dataset.at[i,'Duration between onset and test (detection time) (in hours)'] = onsettest_group[j]
            if j == 0:
                print(i,dataset.at[i,'onset to test'])
            break

0
396 -22
1292 -1
1336 -1
1531 -3
1552 -1
1740 -1
1782 -1
1809 -1
1828 -2
1859 -2
1883 -4
1924 -43082
1928 -1
1954 -2
1971 -43012
1977 -1


In [116]:
dataset.insert(53, 'New vaccination', 0)
for i in range(len(dataset)):
    if dataset.at[i, 'Vaccination'] == '0':
        dataset.at[i, 'New vaccination'] = 0
    elif dataset.at[i, 'Vaccination'] == 'NA':
        dataset.at[i, 'New vaccination'] = 'NA'
    elif dataset.at[i, 'Vaccination'] == '1' or dataset.at[i, 'Vaccination'] == '2' or dataset.at[i, 'Vaccination'] == '3':
        dataset.at[i, 'New vaccination'] = '>=1'

In [117]:
dataset.to_csv('out.csv', index=False)

In [118]:
severe_dataset = dataset.loc[dataset['Clinical classification'] == 1]
severe_dataset.to_csv('severe_out.csv', index=False)